In [1]:
import MySQLdb

db = MySQLdb.connect(  # esto hace la conexion a la base de datos
    host="localhost",  # el puerto es 3308 porque es el puerto que
    user="root",  # le deje a mariaDB
    password="zamromxd",
    database="PokeSearch",
    port=3308,
)
c = db.cursor()

In [2]:
TYPES = dict()
c.execute(f"select * from tipe;")
result = c.fetchall()
for tipe in result:
 TYPES[tipe[0]] = tipe[1]
relations = dict()
for t_id in TYPES:
    if t_id not in relations:
        relations[t_id] = dict()
    c.execute(
        f"select relation,id_type_2 from relation_type where id_type_1={t_id} and relation like '%from' ;"
    )
    temp = c.fetchall()
    for rt in temp:
        if rt[0] not in relations[t_id]:
            relations[t_id][rt[0]] = []
            relations[t_id][rt[0]].append(rt[1])
        else:
            relations[t_id][rt[0]].append(rt[1])


def damages(T: list):
    D = dict()
    for t in T:
        for rt in relations[t]:
            for ti in relations[t][rt]:
                if ti in D:
                    if rt == "double_damage_from":
                        D[ti] *= 2
                    elif rt == "half_damage_from":
                        D[ti] *= 0.5
                    else:
                        D[ti] *= 0
                else:
                    if rt == "double_damage_from":
                        D[ti] = 2
                    elif rt == "half_damage_from":
                        D[ti] = 0.5
                    else:
                        D[ti] = 0
    return D



In [3]:
class Pokemon:
    def __init__(self, name: str):
        c.execute(f"select id,atk,satk,def,sdef,total_stat,name from pokemon where name='{name}';")
        data = c.fetchall()
        self.id = data[0][0]
        self.atk= data[0][1]
        self.defe= data[0][3]
        self.satk= data[0][2]
        self.sdef= data[0][4]
        self.total= data[0][5]
        self.name = data[0][6]
        c.execute(f"select id_type from pok_typ where id_pokemon={data[0][0]};")
        T = c.fetchall()
        self.types = []
        for t in T:
            self.types.append(t[0])
        self.damages = damages(self.types)
        
        self.moves = {"physical": dict(), "status": dict(), "special": dict()}
        c.execute(
            f"select M.name,M.id_type,M.damage_class from move as M inner join pok_mov as PM on  PM.id_move=M.id and PM.id_pokemon={self.id};"
        )
        M = c.fetchall()
        for att in M:
            if att[1] not in self.moves[att[2]]:
                self.moves[att[2]][att[1]] = list()
            self.moves[att[2]][att[1]].append(att[0])

In [4]:
def cantMove(principal:Pokemon):
    cant = dict()
    for clas in principal.moves:
        #c = 0
        cant[clas] = dict()
        for tipo in principal.moves[clas]:
            #c += len(principal.moves[clas][tipo])
            cant[clas][tipo] = len(principal.moves[clas][tipo])
        #cant[clas]['total'] = c
    #print(cant)
    return cant

In [5]:
p = "bulbasaur"
poke = Pokemon(p)
# p = input('Nombre del pokemon: ')
# c.execute(f"select * from pokemon where name='{p}';")
# result = c.fetchall()
pokes = dict()
c.execute(f"select name from pokemon where total_stat>={poke.total-20} and total_stat<={poke.total+20} and id!={poke.id};")
#c.execute(f'select name from pokemon;')
result = c.fetchall()
for r in result:
 pokes[r[0]] = Pokemon(r[0])
 #print(r[0])

In [6]:
def puntaje(M:Pokemon,m_atk,o_def,O:Pokemon):
    ptM = 0
    priCant = cantMove(M)
    for classM in priCant:
        for t in priCant[classM]:
            p = priCant[classM][t]
            pp = p
            if m_atk and classM==m_atk:
                p += pp/4
            if o_def and o_def==classM:
                p -= pp/4
            if t in M.types:
                p += pp / 2
            if t in O.damages:
                p *= O.damages[t]
            ptM +=p
    return ptM

In [7]:
def calcular(main:Pokemon, other: Pokemon):
    if main.atk > main.satk:
        pri_atk = "physical"
    elif main.atk < main.satk:
        pri_atk = "special"
    else:
        pri_atk = False
    if main.defe > main.sdef:
        pri_def = "physical"
    elif main.defe < main.sdef:
        pri_def = "special"
    else:
        pri_def = False
    if other.atk > other.satk:
        other_atk = "physical"
    elif other.atk < other.satk:
        other_atk = "special"
    else:
        other_atk = False
    if other.defe > other.sdef:
        other_def = "physical"
    elif other.defe < other.sdef:
        other_def = "special"
    else:
        other_def = False

    ptM = puntaje(M=main,m_atk=pri_atk,o_def=other_def,O=other)
    ptO = puntaje(M=other, m_atk=other_atk,o_def=pri_def,O=main)
    
    return ptM-ptO + (main.total-other.total)

In [8]:
calcular(poke, pokes["pikachu"])

1.125

In [11]:
def pokeSearch(p:str):
    poke = Pokemon(p)
    pokes = dict()
    c.execute(
        f"select name from pokemon where total_stat>={poke.total-20} and total_stat<={poke.total+20} and id!={poke.id};"
    )
    data = c.fetchall()
    for r in data:
        pokes[r[0]] = Pokemon(r[0])
    PL = list()
    for pp in pokes:
        PL.append((pp, calcular(poke, pokes[pp])))
        PL.sort(key=lambda x: x[1])
    print(f"{p}:")
    print(f"\tWorst against:  {[p[0] for p in PL[0:3]]}")
    print(f"\tBetter against: {[p[0] for p in PL[-3:]]}")
    return [p[0] for p in PL[0:3]], [p[0] for p in PL[-3:]]

In [9]:
PL = list()
for pp in pokes:
    PL.append((pp, calcular(poke, pokes[pp])))
    PL.sort(key=lambda x: x[1])
print(f"{p}:")
print(f"\tWorst against:  {[p[0] for p in PL[0:3]]}")
print(f"\tBetter against: {[p[0] for p in PL[-3:]]}")

bulbasaur:
	Worst against:  ['delibird', 'drowzee', 'natu']
	Better against: ['binacle', 'sobble', 'larvitar']


In [10]:
for p in pokes:
    poke = Pokemon(p)
    PL = list()
    for pp in pokes:
        PL.append((pp, calcular(poke, pokes[pp])))
    # print(PL)
    PL.sort(key=lambda x: x[1])
    #print(f'{p}\t{PL}')
    print(f'{p}:')
    print(f'\tBetter: {PL[0:3]}')
    print(f'\tWorst: {PL[-3:]}')

charmander:
	Better: [('frillish', -72.75), ('clefairy', -72.25), ('squirtle', -67.625)]
	Worst: [('snorunt', 112.125), ('snover', 117.5), ('sewaddle', 134.875)]
squirtle:
	Better: [('pumpkaboo-average', -62.625), ('clefairy', -59.625), ('phantump', -59.5)]
	Worst: [('geodude', 144.75), ('numel', 150.375), ('larvitar', 171.125)]
pikachu:
	Better: [('golett', -58.625), ('sandygast', -58.25), ('pumpkaboo-average', -52.0)]
	Worst: [('unown', 108.25), ('sobble', 115.875), ('ducklett', 170.625)]
sandshrew:
	Better: [('frillish', -90.25), ('pumpkaboo-average', -81.25), ('clefairy', -74.0)]
	Worst: [('unown', 91.5), ('beldum', 94.25), ('salandit', 95.9375)]
clefairy:
	Better: [('frillish', -1.5), ('clefairy', 0.0), ('honedge', 4.0625)]
	Worst: [('croagunk', 158.1875), ('deino', 164.5), ('clobbopus', 177.875)]
vulpix:
	Better: [('psyduck', -91.25), ('clefairy', -89.25), ('squirtle', -88.0)]
	Worst: [('meltan', 72.0), ('beldum', 92.625), ('sewaddle', 96.625)]
oddish:
	Better: [('drowzee', -93.2

In [12]:
p = 'charmander'
pokeSearch(p)

charmander:
	Worst against:  ['clefairy', 'squirtle', 'psyduck']
	Better against: ['snorunt', 'steenee', 'sewaddle']
